## 推移確率行列の作成関数

In [1]:
import pandas as pd
import numpy as np

In [28]:
popularity = pd.read_csv('./csv/popularity_33_2_500_20_76_200.csv', engine='python', encoding='utf-8', header=None)
N = len(popularity.values.tolist())
R = len(popularity.values.tolist()[0])
K_total = 200
popularity.values.tolist()

[[16, 4],
 [4, 5],
 [6, 5],
 [7, 16],
 [3, 4],
 [4, 4],
 [5, 8],
 [2, 5],
 [6, 6],
 [14, 3],
 [7, 2],
 [5, 11],
 [7, 6],
 [6, 16],
 [7, 5],
 [4, 6],
 [8, 5],
 [5, 5],
 [3, 4],
 [3, 5],
 [3, 5],
 [4, 7],
 [3, 2],
 [5, 6],
 [6, 13],
 [8, 7],
 [3, 4],
 [4, 7],
 [17, 3],
 [7, 13],
 [18, 18],
 [19, 4],
 [14, 17]]

In [19]:
distance_matrix = pd.read_csv('./csv/distance_33_2_500_20_76_200.csv', engine='python', encoding='utf-8', header=None)
distance_matrix.values

array([[  0.     , 124.7758 , 485.02577, ..., 289.83616, 230.78345,
        289.84996],
       [124.7758 ,   0.     , 378.84166, ..., 250.71099, 198.17669,
        166.64333],
       [485.02577, 378.84166,   0.     , ..., 283.95246, 312.64197,
        237.11811],
       ...,
       [289.83616, 250.71099, 283.95246, ...,   0.     ,  59.07622,
        256.90854],
       [230.78345, 198.17669, 312.64197, ...,  59.07622,   0.     ,
        235.28706],
       [289.84996, 166.64333, 237.11811, ..., 256.90854, 235.28706,
          0.     ]])

In [21]:
#重力モデルで推移確率行列を作成 
def getGravity(distance, popularity): #distanceは距離行列、popularityはクラス分の人気度
    C = 0.1475
    alpha = 1.0
    beta = 1.0
    eta = 0.5
    class_number = len(popularity[0]) #クラス数
    tp = np.zeros((len(distance) * class_number, len(distance) * class_number))
    for r in range(class_number):
        for i in range(len(distance) * r, len(distance) * (r+1)):
            for j in range(len(distance) * r, len(distance) * (r+1)):
                if distance[i % len(distance)][j % len(distance)] > 0:
                    tp[i][j] = C * (popularity[i % len(distance)][r]**alpha) * (popularity[j % len(distance)][r]**beta) / (distance[i % len(distance)][j % len(distance)]**eta)
    row_sum = np.sum(tp, axis=1) #行和を算出
    for i in range(len(tp)): #行和を1にする
        if row_sum[i] > 0:
            tp[i] /= row_sum[i]
    return tp 

In [29]:
p = getGravity(distance_matrix.values, popularity.values.tolist())
np.savetxt('./Result/inital_p_'+str(N)+'_'+str(R)+'_'+str(K_total)+'.csv', p, delimiter=',', fmt='%.5f')
#saveCSV(transitionprob, '/content/drive/MyDrive/研究/BCMP/TransitionProbability/csv/transition_test.csv')
p

array([[0.        , 0.02461401, 0.01872649, ..., 0.        , 0.        ,
        0.        ],
       [0.0858907 , 0.        , 0.01848476, ..., 0.        , 0.        ,
        0.        ],
       [0.05547021, 0.0156911 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.03558781,
        0.07252827],
       [0.        , 0.        , 0.        , ..., 0.13504199, 0.        ,
        0.06390761],
       [0.        , 0.        , 0.        , ..., 0.07079332, 0.0164388 ,
        0.        ]])

In [30]:
#MVAファイルの取り込み
#import sys
#sys.path.append('/content/drive/MyDrive/研究/BCMP/')
import BCMP_MVA as mdl

In [31]:
#MVAの実施
import time
#N = 33 #33
#R = 2
K_total = 200
K = [(K_total + i) // R for i in range(R)] #クラス人数を自動的に配分する
mu = np.full((R, N), 1) #サービス率を同じ値で生成(サービス率は調整が必要)
type_list = np.full(N, 1) #サービスタイプはFCFS
m = np.full(N, 1) #今回は窓口数1(複数窓口は実装できていない)
bcmp = mdl.BCMP_MVA(N, R, K, mu, type_list, p, m)
start = time.time()
L = bcmp.getMVA()
elapsed_time = time.time() - start
print ("calclation_time:{0}".format(elapsed_time) + "[sec]")
print('L = \n{0}'.format(L))
np.savetxt('./Result/inital_L_'+str(N)+'_'+str(R)+'_'+str(K_total)+'.csv', L, delimiter=',', fmt='%.5f')

calclation_time:30.6965594291687[sec]
L = 
[[9.41037909e-01 2.91643009e-01]
 [1.68819091e-01 2.24089093e-01]
 [1.94530120e-01 1.68480254e-01]
 [5.98171528e-01 1.30167023e+00]
 [1.11747308e-01 1.52387673e-01]
 [1.56028202e-01 1.59408832e-01]
 [1.79223177e-01 2.90592192e-01]
 [7.04133929e-02 1.79190426e-01]
 [2.94148870e-01 2.93696452e-01]
 [7.87675230e-01 1.82000073e-01]
 [2.44386002e-01 7.20198850e-02]
 [2.87457993e-01 5.83762174e-01]
 [3.12677851e-01 2.81009263e-01]
 [4.36150233e-01 1.06493493e+00]
 [3.23200699e-01 2.37009195e-01]
 [1.79982049e-01 2.83241757e-01]
 [4.51996280e-01 2.90697324e-01]
 [2.18306035e-01 2.29574068e-01]
 [1.46656143e-01 2.02894551e-01]
 [1.13431169e-01 1.78568155e-01]
 [1.32855629e-01 2.29015636e-01]
 [2.32664408e-01 4.02876834e-01]
 [1.14962544e-01 7.36361489e-02]
 [2.24580658e-01 2.61942314e-01]
 [2.34323179e-01 5.04075594e-01]
 [5.06852580e-01 4.65284537e-01]
 [1.15436632e-01 1.57118913e-01]
 [1.48774912e-01 2.60479989e-01]
 [9.80968295e-01 1.93194593e-01]
